<a href="https://colab.research.google.com/github/connect-midhunr/online-retail-customer-segmentation-RFM-analysis/blob/main/Online_Retail_Customer_Segmentation_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Extraction/identification of major topics & themes discussed in news articles. </u></b>

## <b> Problem Description </b>

### In this project, your task is to identify major customer segments on a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

## <b> Data Description </b>

### <b>Attribute Information: </b>

* ### InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
* ### StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
* ### Description: Product (item) name. Nominal.
* ### Quantity: The quantities of each product (item) per transaction. Numeric.
* ### InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
* ### UnitPrice: Unit price. Numeric, Product price per unit in sterling.
* ### CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
* ### Country: Country name. Nominal, the name of the country where each customer resides.

# Business Task

Analyse the transactions data of a non-store online retail company and build a machine learning model to identify major customer segments.

# Importing Libraries

In [1]:
# importing all relevant python libraries
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Reading Data

In [2]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# reading data and storing it in a dataframe
sales_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/AlmaBetter/Capstone Projects/Unsupervised Machine Learning/Online Retail Customer Segmentation - Midhun R/Online Retail.xlsx')

# Data Inspection

The given dataset contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. Lets first examine the data present in it.

In [6]:
# exploring the head of the dataframe
sales_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
# exploring the tail of the dataframe
sales_df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [8]:
# brief summary of dataframe
sales_df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [10]:
# total number of rows in the dataset
print(f"Total number of rows: {sales_df.shape[0]}")

# number of duplicate rows
print(f"Number of duplicate rows: {sales_df[sales_df.duplicated()].shape[0]}")

Total number of rows: 541909
Number of duplicate rows: 5268


Out of the 541909 rows in the dataset, 5268 rows are duplicates.

In [11]:
# exploring the columns of the dataframe
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


The dataset have 8 features. Two of them have missing values and one needs datatype conversion.

In [13]:
# unique values in each column of the dataframe
print(sales_df.apply(lambda col: col.unique()))

InvoiceNo      [536365, 536366, 536367, 536368, 536369, 53637...
StockCode      [85123A, 71053, 84406B, 84029G, 84029E, 22752,...
Description    [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
Quantity       [6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 80...
InvoiceDate    [2010-12-01T08:26:00.000000000, 2010-12-01T08:...
UnitPrice      [2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2.1...
CustomerID     [17850.0, 13047.0, 12583.0, 13748.0, 15100.0, ...
Country        [United Kingdom, France, Australia, Netherland...
dtype: object


# Data Cleaning

Let's take a copy of the dataset and work on it so that the original data don't get modified.

In [15]:
# copying the dataset
work_df = sales_df.copy()
work_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


## Removing Duplicate Rows

In [16]:
# removing duplicate rows
work_df.drop_duplicates(inplace=True)

In [18]:
# total number of rows in the dataset
print(f"Total number of rows: {work_df.shape[0]}")

# number of duplicate rows
print(f"Number of duplicate rows: {work_df[work_df.duplicated()].shape[0]}")

# percentage of rows removed
print(f"Percentage of reduction in the number of rows: {round((sales_df.shape[0]-work_df.shape[0])/sales_df.shape[0]*100, 2)}%")

Total number of rows: 536641
Number of duplicate rows: 0
Percentage of reduction in the number of rows: 0.97%


After removing duplicate rows, number of rows has been reduced from 541909 to 536641, a reduction of 0.97%.

## Handling Missing Values

In [21]:
# defining a function to return the count and percentage of missing values in a dataset
def get_missing_values_count_and_percentage(dataframe):
  num = 0
  for feature in dataframe.columns:
    count = dataframe[feature].isnull().sum()
    percentage = round(count/dataframe.shape[0]*100, 2)
    if count > 0:
      num += 1
      print(f"{feature}: {count}({percentage}%)")
  if num == 0:
    print("No missing values in the dataframe")

In [22]:
# number and percentage of missing values in each feature if present
get_missing_values_count_and_percentage(work_df)

Description: 1454(0.27%)
CustomerID: 135037(25.16%)


Let's drop Description from the dataset as it is not important for customer segmentation.

In [27]:
# dropping Description from the dataset
work_df.drop('Description', inplace=True, axis=1)
work_df.columns

# features in the dataset
for feature in work_df.columns:
  print(feature)

InvoiceNo
StockCode
Quantity
InvoiceDate
UnitPrice
CustomerID
Country


In the case of CustomerID, let's check if InvoiceNo in the rows having null CustomerID is present in other rows, so that CustomerID in those rows can be used for imputing the missing values.

In [35]:
# InvoiceNo for null CustomerID
invoiceNo_for_null_custometId = set(work_df[work_df['CustomerID'].isnull()]['InvoiceNo'])

# CustomerID for InvoiceNo of null CustomerID
work_df[(work_df['InvoiceNo'].isin(invoiceNo_for_null_custometId)) & (work_df['CustomerID'].notnull())]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


InvoiceNo in the rows having null CustomerID is found to be not present in other rows. 

Since CustomerID is important for customer segmentation, rows where CustomerID is null must be removed for accurate results. But one-fourth of the dataset have missing values in CustomerID and removing it causes a huge loss of information. 

So, let's assume that these rows represent new customers as CustomerID should be present for existing customers. Missing values can be imputed by generating new CustomerIDs.

In [69]:
# generating CustomerID for new customers
new_customerID = list(range(int(work_df['CustomerID'].max()) + 1, int(work_df['CustomerID'].max()) + 1 + len(invoiceNo_for_null_custometId)))

# clubbing InvoiceNo and new customer IDs into tuples
new_customerID_for_invoiceNo = dict(zip(invoiceNo_for_null_custometId, new_customerID))

# imputing missing values in CustomerID
for index, row in work_df.iterrows():
  if ((row['InvoiceNo'] in new_customerID_for_invoiceNo) & (row['CustomerID'].isnull())):
    work_df['CustomerID'] = new_customerID_for_invoiceNo[row['InvoiceNo']]

Streaming output truncated to the last 5000 lines.
          ...  
541904    21939
541905    21939
541906    21939
541907    21939
541908    21939
Name: CustomerID, Length: 536641, dtype: int64
541109
541109 581492 0         21939
1         21939
2         21939
3         21939
4         21939
          ...  
541904    21939
541905    21939
541906    21939
541907    21939
541908    21939
Name: CustomerID, Length: 536641, dtype: int64
541110
541110 581492 0         21939
1         21939
2         21939
3         21939
4         21939
          ...  
541904    21939
541905    21939
541906    21939
541907    21939
541908    21939
Name: CustomerID, Length: 536641, dtype: int64
541111
541111 581492 0         21939
1         21939
2         21939
3         21939
4         21939
          ...  
541904    21939
541905    21939
541906    21939
541907    21939
541908    21939
Name: CustomerID, Length: 536641, dtype: int64
541112
541112 581492 0         21939
1         21939
2         21939
3    

In [70]:
work_df

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,6,2010-12-01 08:26:00,2.55,21942,United Kingdom
1,536365,71053,6,2010-12-01 08:26:00,3.39,21942,United Kingdom
2,536365,84406B,8,2010-12-01 08:26:00,2.75,21942,United Kingdom
3,536365,84029G,6,2010-12-01 08:26:00,3.39,21942,United Kingdom
4,536365,84029E,6,2010-12-01 08:26:00,3.39,21942,United Kingdom
...,...,...,...,...,...,...,...
541904,581587,22613,12,2011-12-09 12:50:00,0.85,21942,France
541905,581587,22899,6,2011-12-09 12:50:00,2.10,21942,France
541906,581587,23254,4,2011-12-09 12:50:00,4.15,21942,France
541907,581587,23255,4,2011-12-09 12:50:00,4.15,21942,France


In [71]:
# number and percentage of missing values in each feature if present
get_missing_values_count_and_percentage(work_df)

No missing values in the dataframe


In [68]:
work_df[work_df['CustomerID'].isnull()]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...
541536,581498,85099B,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom
